In [1]:
!pip install pandas nltk scikit-learn

  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.14.0 requires numpy>=1.23.5, but you have numpy 1.22.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.


In [47]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [10]:
amazon_data = pd.read_csv('C:/Users/Alexander Bryan/sentiment labelled sentences/amazon_cells_labelled.csv', header=None, names=['Sentence', 'Label', '1', '2', '3', '4'], delimiter=None)
amazon_data.head()

,Sentence,Label,1,2,3,4
0,So there is no way for me to plug it in here i...,0,NaN,NaN,NaN,NaN
1,Good case,Excellent value.,1,NaN,NaN,NaN
2,Great for the jawbone.,1,NaN,NaN,NaN,NaN
3,Tied to charger for conversations lasting more...,0,NaN,NaN,NaN,NaN
4,The mic is great.,1,NaN,NaN,NaN,NaN


In [11]:
imdb_data = pd.read_csv('C:/Users/Alexander Bryan/sentiment labelled sentences/imdb_labelled.csv', header=None, names=['Sentence', 'Label', '1', '2', '3', '4', '5' , '6'], delimiter=None)
imdb_data.head()

,Sentence,Label,1,2,3,4,5,6
0,A very,very,very slow-moving,aimless movie about a distressed,drifting young man.,0,NaN,NaN
1,Not sure who was more lost - the flat characte...,nearly half of whom walked out.,0,NaN,NaN,NaN,NaN,NaN
2,Attempting artiness with black & white and cle...,the movie disappointed - became even more rid...,0,NaN,NaN,NaN,NaN,NaN
3,Very little music or anything to speak of.,0,NaN,NaN,NaN,NaN,NaN,NaN
4,The best scene in the movie was when Gerardo i...,1,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
yelp_data = pd.read_csv('C:/Users/Alexander Bryan/sentiment labelled sentences/yelp_labelled.csv', header=None, names=['Sentence', 'Label', '1', '2', '3', '4'], delimiter=None)
yelp_data.head()

,Sentence,Label,1,2,3,4
0,Wow... Loved this place.,1,NaN,NaN,NaN,NaN
1,Crust is not good.,0,NaN,NaN,NaN,NaN
2,Not tasty and the texture was just nasty.,0,NaN,NaN,NaN,NaN
3,Stopped by during the late May bank holiday of...,1,NaN,NaN,NaN,NaN
4,The selection on the menu was great and so wer...,1,NaN,NaN,NaN,NaN


In [19]:
combined_data = pd.DataFrame({
    'Amazon': amazon_data.apply(lambda x: ' '.join([str(val) for val in x.dropna()]), axis=1),
    'IMDB': imdb_data.apply(lambda x: ' '.join([str(val) for val in x.dropna()]), axis=1),
    'Yelp': yelp_data.apply(lambda x: ' '.join([str(val) for val in x.dropna()]), axis=1)
})

In [20]:
combined_data.columns = ['Amazon', 'IMDB', 'Yelp']

In [21]:
combined_data.head()

,Amazon,IMDB,Yelp
0,So there is no way for me to plug it in here i...,A very very very slow-moving aimless movie ...,Wow... Loved this place. 1
1,Good case Excellent value. 1,Not sure who was more lost - the flat characte...,Crust is not good. 0
2,Great for the jawbone. 1,Attempting artiness with black & white and cle...,Not tasty and the texture was just nasty. 0
3,Tied to charger for conversations lasting more...,Very little music or anything to speak of. 0,Stopped by during the late May bank holiday of...
4,The mic is great. 1,The best scene in the movie was when Gerardo i...,The selection on the menu was great and so wer...


In [32]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Ubah teks menjadi lowercase
        text = ' '.join([word for word in word_tokenize(text) if word.isalnum()])  # Hapus karakter non-alphanumeric
        return text
    else:
        return np.nan

# Menerapkan fungsi pada kolom 'Amazon', 'IMDB', dan 'Yelp'
combined_data['Cleaned Amazon'] = combined_data['Amazon'].apply(clean_text)
combined_data['Cleaned IMDB'] = combined_data['IMDB'].apply(clean_text)
combined_data['Cleaned Yelp'] = combined_data['Yelp'].apply(clean_text)

# Menggabungkan hasil pembersihan ke dalam kolom baru 'Cleaned Sentence'
combined_data['Cleaned Sentence'] = combined_data['Cleaned Amazon'] + ' ' + combined_data['Cleaned IMDB'] + ' ' + combined_data['Cleaned Yelp']

# Memilih kolom yang diperlukan
combined_data_cleaned = combined_data[['Cleaned Amazon', 'Cleaned IMDB', 'Cleaned Yelp']]
combined_data_cleaned.head()

,Cleaned Amazon,Cleaned IMDB,Cleaned Yelp
0,so there is no way for me to plug it in here i...,a very very very aimless movie about a distres...,wow loved this place 1
1,good case excellent value 1,not sure who was more lost the flat characters...,crust is not good 0
2,great for the jawbone 1,attempting artiness with black white and cleve...,not tasty and the texture was just nasty 0
3,tied to charger for conversations lasting more...,very little music or anything to speak of 0,stopped by during the late may bank holiday of...
4,the mic is great 1,the best scene in the movie was when gerardo i...,the selection on the menu was great and so wer...


In [45]:
# Fungsi untuk tokenisasi
def tokenize_text(text):
    if isinstance(text, str):
        tokens = word_tokenize(text)
        return tokens
    else:
        return []

# Menerapkan fungsi pada kolom 'Amazon', 'IMDB', dan 'Yelp'
combined_data['Tokenized Amazon'] = combined_data['Cleaned Amazon'].apply(tokenize_text)
combined_data['Tokenized IMDB'] = combined_data['Cleaned IMDB'].apply(tokenize_text)
combined_data['Tokenized Yelp'] = combined_data['Cleaned Yelp'].apply(tokenize_text)

# Menggabungkan hasil tokenisasi ke dalam kolom baru 'Tokenized Sentence'
combined_data['Tokenized Sentence'] = combined_data.apply(lambda row: ' '.join(row['Tokenized Amazon'] + row['Tokenized IMDB'] + row['Tokenized Yelp']), axis=1)

# Memilih kolom yang diperlukan
combined_data_tokenized = combined_data[['Tokenized Amazon', 'Tokenized IMDB', 'Tokenized Yelp']]
combined_data_tokenized.head()

,Tokenized Amazon,Tokenized IMDB,Tokenized Yelp
0,"[so, there, is, no, way, for, me, to, plug, it...","[a, very, very, very, aimless, movie, about, a...","[wow, loved, this, place, 1]"
1,"[good, case, excellent, value, 1]","[not, sure, who, was, more, lost, the, flat, c...","[crust, is, not, good, 0]"
2,"[great, for, the, jawbone, 1]","[attempting, artiness, with, black, white, and...","[not, tasty, and, the, texture, was, just, nas..."
3,"[tied, to, charger, for, conversations, lastin...","[very, little, music, or, anything, to, speak,...","[stopped, by, during, the, late, may, bank, ho..."
4,"[the, mic, is, great, 1]","[the, best, scene, in, the, movie, was, when, ...","[the, selection, on, the, menu, was, great, an..."


In [49]:
# Membuat sebuah instance dari Porter stemmer
porter = PorterStemmer()

# Fungsi untuk stemming
def stem_text(tokens):
    return [porter.stem(token) for token in tokens]

# Menerapkan fungsi pada kolom 'Amazon', 'IMDB', dan 'Yelp'
combined_data['Stemmed Amazon'] = combined_data['Tokenized Amazon'].apply(stem_text)
combined_data['Stemmed IMDB'] = combined_data['Tokenized IMDB'].apply(stem_text)
combined_data['Stemmed Yelp'] = combined_data['Tokenized Yelp'].apply(stem_text)

# Menggabungkan hasil stemming ke dalam kolom baru 'Stemmed Sentence'
combined_data['Stemmed Sentence'] = combined_data.apply(lambda row: ' '.join(row['Stemmed Amazon'] + row['Stemmed IMDB'] + row['Stemmed Yelp']), axis=1)

# Memilih kolom yang diperlukan
combined_data_stemmed = combined_data[['Stemmed Amazon', 'Stemmed IMDB', 'Stemmed Yelp']]
combined_data_stemmed.head()

,Stemmed Amazon,Stemmed IMDB,Stemmed Yelp
0,"[so, there, is, no, way, for, me, to, plug, it...","[a, veri, veri, veri, aimless, movi, about, a,...","[wow, love, thi, place, 1]"
1,"[good, case, excel, valu, 1]","[not, sure, who, wa, more, lost, the, flat, ch...","[crust, is, not, good, 0]"
2,"[great, for, the, jawbon, 1]","[attempt, arti, with, black, white, and, cleve...","[not, tasti, and, the, textur, wa, just, nasti..."
3,"[tie, to, charger, for, convers, last, more, t...","[veri, littl, music, or, anyth, to, speak, of, 0]","[stop, by, dure, the, late, may, bank, holiday..."
4,"[the, mic, is, great, 1]","[the, best, scene, in, the, movi, wa, when, ge...","[the, select, on, the, menu, wa, great, and, s..."


In [50]:
# Fungsi untuk filtering stop words
def filter_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]

# Menerapkan fungsi pada kolom 'Amazon', 'IMDB', dan 'Yelp'
combined_data['Filtered Amazon'] = combined_data['Stemmed Amazon'].apply(filter_stop_words)
combined_data['Filtered IMDB'] = combined_data['Stemmed IMDB'].apply(filter_stop_words)
combined_data['Filtered Yelp'] = combined_data['Stemmed Yelp'].apply(filter_stop_words)

# Menggabungkan hasil filtering ke dalam kolom baru 'Filtered Sentence'
combined_data['Filtered Sentence'] = combined_data.apply(lambda row: ' '.join(row['Filtered Amazon'] + row['Filtered IMDB'] + row['Filtered Yelp']), axis=1)

# Memilih kolom yang diperlukan
combined_data_filtered = combined_data[['Filtered Amazon', 'Filtered IMDB', 'Filtered Yelp']]
combined_data_filtered.head()

,Filtered Amazon,Filtered IMDB,Filtered Yelp
0,"[way, plug, us, unless, go, convert, 0]","[veri, veri, veri, aimless, movi, distress, dr...","[wow, love, thi, place, 1]"
1,"[good, case, excel, valu, 1]","[sure, wa, lost, flat, charact, audienc, nearl...","[crust, good, 0]"
2,"[great, jawbon, 1]","[attempt, arti, black, white, clever, camera, ...","[tasti, textur, wa, nasti, 0]"
3,"[tie, charger, convers, last, 45, problem, 0]","[veri, littl, music, anyth, speak, 0]","[stop, dure, late, may, bank, holiday, rick, s..."
4,"[mic, great, 1]","[best, scene, movi, wa, gerardo, tri, find, so...","[select, menu, wa, great, price, 1]"


In [51]:
# Membuat fungsi untuk memberikan label sentimen
def label_sentiment(text):
    # Misalnya, kita akan anggap sentimen positif jika terdapat kata 'good' dan sentimen negatif jika terdapat kata 'bad'
    if 'good' in text:
        return 'Positive'
    elif 'bad' in text:
        return 'Negative'
    else:
        return 'Neutral'

# Menerapkan fungsi pada kolom 'Filtered Sentence' untuk memberikan label sentimen
combined_data['Sentiment'] = combined_data['Filtered Sentence'].apply(label_sentiment)

# Menampilkan hasil analisis sentimen
sentiment_counts = combined_data['Sentiment'].value_counts()
print(sentiment_counts)

Neutral     813
Positive    159
Negative     28
Name: Sentiment, dtype: int64


In [52]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [53]:
# Membagi data menjadi data latih dan data uji
train_data, test_data, train_labels, test_labels = train_test_split(combined_data['Filtered Sentence'], combined_data['Sentiment'], test_size=0.2, random_state=42)

In [54]:
# Menggunakan CountVectorizer untuk mengonversi teks menjadi vektor fitur
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data)
X_test = vectorizer.transform(test_data)

In [55]:
# Membuat dan melatih model Naive Bayes
model = MultinomialNB()
model.fit(X_train, train_labels)

MultinomialNB()

In [56]:
# Menguji model pada data uji
predictions = model.predict(X_test)

In [57]:
# Evaluasi performa model
accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.875
Classification Report:
               precision    recall  f1-score   support

    Negative       1.00      0.14      0.25         7
     Neutral       0.88      0.99      0.93       152
    Positive       0.86      0.59      0.70        41

    accuracy                           0.88       200
   macro avg       0.91      0.57      0.62       200
weighted avg       0.88      0.88      0.86       200



In [58]:
combined_data.to_csv("C:/Users/Alexander Bryan/sentiment labelled sentences/combined_processed_data.csv", index=False)